In [1]:
versioninfo()

Julia Version 1.10.4
Commit 48d4fd4843 (2024-06-04 10:41 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: 24 × 12th Gen Intel(R) Core(TM) i9-12900K
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, alderlake)
Threads: 8 default, 0 interactive, 4 GC (on 24 virtual cores)
Environment:
  JULIA_NUM_THREADS = 8


In [2]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
Pkg.status()

  Activating project at `C:\Users\o6m1g\Documents\GitHub\aSMC-CV`


Status `C:\Users\o6m1g\Documents\GitHub\aSMC-CV\Project.toml`
  [0bf59076] AdvancedHMC v0.8.0
  [336ed68f] CSV v0.10.15
  [a93c6f00] DataFrames v1.7.0
  [31c24e10] Distributions v0.25.120
  [4138dd39] JLD v0.13.5
  [b964fa9f] LaTeXStrings v1.4.0
  [6fdf6af0] LogDensityProblems v2.1.2
  [86f7a689] NamedArrays v0.10.4
  [ce719bf2] PSIS v0.9.8
  [91a5bcdd] Plots v1.40.16
  [92933f4c] ProgressMeter v1.10.4
  [f2b01f46] Roots v2.2.8
  [2913bbd2] StatsBase v0.34.5
  [4c63d2b9] StatsFuns v1.5.0
  [f3b207a7] StatsPlots v0.15.7
  [fce5fe82] Turing v0.39.4
  [ade2ca70] Dates
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random


# Setup

In [3]:
include("Init.jl")

In [4]:
"Running on ", Threads.nthreads(), " threads."

("Running on ", 8, " threads.")

In [5]:
using Turing

In [6]:
RUN_MCMC = false
RUN_aSMC = true;

In [7]:
# Tag by timestamp
SESSION = Dates.format(now(), "yyyy-mm-dd-HH-MM-SS");

# Data

In [6]:
# Load data
df = CSV.read(joinpath("data", "radon", "radon.csv"), DataFrame)
first(df, 4)

Row,log_radon,floor,log_u,county
,Float64,Int64,Float64,String31
1,1.43508,0,0.165862,MARTIN
2,1.02962,0,-0.418054,RAMSEY
3,0.262364,0,-0.418054,RAMSEY
4,1.28093,0,-0.418054,RAMSEY


In [7]:
struct Data_Radon
    y::Vector{Float64}    # Response
    X::Matrix{Float64}    # Observation-level covariates
    U::Matrix{Float64}    # County-level covariates
    county::Vector{Int64} # County index for each observation (1 to G)
    N::Int64              # Total number of observations
    G::Int64              # Number of unique counties
    N_g::Vector{Int64}    # Observation count per county (length G)
end

In [8]:
# Construct data for "radon" example
let
    y = df.log_radon # Response variable
    X = [ones(size(df, 1)) df.floor] # Covariates
    N = length(y) # Total number of observations
    G = length(unique(df.county)) # Total number of unique counties

    # County-level covariate
    u = unique(df.log_u)
    @assert length(u) == G == 85 # Assert expected structure: exactly 85 counties and log_u values
    U = [ones(G) u]

    # Assign county index to each observation
    county_dict = Dict(g => g_i for (g_i, (g, _)) in enumerate(zip(unique(df.county), u)))
    county = [county_dict[g] for g in df.county]

    # Count observations per county → vector of length G
    N_g = sum(df.county .== permutedims(unique(df.county)); dims=1) |> vec

    # Assemble data structure for modeling
    global data = Data_Radon(y, X, U, county, N, G, N_g)
end

Data_Radon([1.4350845252893225, 1.0296194171811583, 0.26236426446749106, 1.2809338454620642, 1.7227665977411035, 1.7227665977411035, 0.26236426446749106, 1.6094379124341003, 1.410986973710262, 1.2809338454620642  …  1.3350010667323402, -0.5108256237659907, 0.09531017980432493, 0.4054651081081644, -0.6931471805599453, -0.5108256237659907, 0.5306282510621705, 0.0, 2.2192034840549946, 0.8329091229351041], [1.0 0.0; 1.0 0.0; … ; 1.0 1.0; 1.0 1.0], [1.0 0.16586183575526953; 1.0 -0.418053510614464; … ; 1.0 -0.7518722327017873; 1.0 -0.6633476306014291], [1, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  56, 13, 51, 52, 13, 13, 13, 13, 15, 82], 919, 85, [7, 32, 5, 3, 3, 6, 4, 4, 2, 11  …  12, 6, 5, 4, 4, 3, 4, 5, 3, 11])

# MCMC

In [9]:
@model function Model(
        data::Data_Radon,
        leave_g::Int64,
        ϕ::Union{Float64, Int64},
    )
    Γ ~ filldist(Normal(), 2, 2)
    v_y ~ InverseGamma(2, 1)
    v_β ~ filldist(Exponential(), 2)
    
    β = Vector(undef, data.G)
    for g in 1:data.G
        β[g] ~ MvNormal(Γ * data.U[g,:], v_β |> Diagonal)
    end
    
    for i in 1:data.N
        # Apply leave-one-group-out logic:
        # if observation belongs to leave_g, downweight likelihood by (1 - ϕ)
        _power = data.county[i] == leave_g ? (1 - ϕ) : 1
        Turing.@addlogprob! _power * logpdf(
            Normal(data.X[i,:]' * β[data.county[i]], v_y |> sqrt), data.y[i])
    end
    
    (; Γ, v_β, v_y, β)
end

Model (generic function with 2 methods)

In [10]:
function run_MCMC(; leave_g::Int64, save_bool::Bool=false)
    """
    Function to run MCMC, leaving (leave_g)-th group out.
    Set leave_g = -1 to target baseline posterior.
    """
    # Instantiate baseline posterior
    @assert leave_g ∈ [-1; 1:data.G]
    _model = nothing
    if leave_g == -1
        # leave_g = -1 ⟺ no leave-group-out
        # ϕ = 0 ⟺ full weight
        _model = Model(data, -1, 0)
    else
        # ϕ = 1 ⟺ no weight; case deleted
        _model = Model(data, leave_g, 1)
    end
    ℓπ = LogDensityFunction(_model)
    Turing.DynamicPPL.link!!(ℓπ.varinfo, _model)
    
    D = LogDensityProblems.dimension(ℓπ) # Dimensionality of parameter space
    metric = DiagEuclideanMetric(D)
    hamiltonian = Hamiltonian(metric, ℓπ)

    # Sampling settings
    n_burn, n_adapts, n_samples, thin = 1_000, 1_000, 3_000, 3 # By pilot run: abs autocorr is under 0.05 after 3 for majority of parameters
    initial_θ = ones(D)
    initial_ϵ = find_good_stepsize(hamiltonian, initial_θ)
    integrator = Leapfrog(initial_ϵ)
    kernel = HMCKernel(Trajectory{MultinomialTS}(integrator, GeneralisedNoUTurn()))
    adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.8, integrator))
    
    time = @elapsed samples, stats = sample(
        hamiltonian, kernel, initial_θ, n_samples + n_burn, adaptor, n_adapts;
        progress=true)

    # Post-process samples:
    # - drop burn-in;
    # - apply thinning (every 3rd)
    # - invert bijector, recover params in original space
    chain = samples[n_burn+1:thin:end] .|> Turing.inverse(Turing.bijector(_model))

    # Save output
    if save_bool
        _dir = joinpath("output_$(SESSION)", "radon", "mcmc")
        isdir(_dir) || mkpath(_dir) # Ensure path is present
        if leave_g == -1
            save(joinpath(_dir, "chain.jld"), "data", chain)
            CSV.write(joinpath(_dir, "stats_df.csv"), stats |> DataFrame)
        else
            save(joinpath(_dir, "chain_lgo-$(leave_g).jld"), "data", chain)
        end
    end
    
    (; chain, stats, time)
end

run_MCMC (generic function with 1 method)

## Initial draw

In [10]:
Random.seed!(1)
RUN_MCMC && (; chain, stats, time) = run_MCMC(leave_g=-1)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter C:\Users\o6m1g\.julia\packages\ProgressMeter\kniaW\src\ProgressMeter.jl:607
Sampling 100%|███████████████████████████████| Time: 0:15:18
                                    iterations: 4000
                   ratio_divergent_transitions: 0.0
   ratio_divergent_transitions_during_adaption: 0.01
                                       n_steps: 31
                                     is_accept: true
                               acceptance_rate: 0.6337821096759917
                                   log_density: -1011.6894998218688
                            hamiltonian_energy: 1116.623272388024
                      hamiltonian_energy_error: 0.38121541874329523

## Naive LGO

In [12]:
Random.seed!(1)

if RUN_MCMC
    
    times_radon = Float64[] # Runtime for each MCMC-LGO run
    
    # Loop over each group for bruteforce MCMC-LGO
    for leave_g in 1:data.G
        @info "Leaving g=$(leave_g) out" 
        
        (; chain, stats, time) = run_MCMC(leave_g=leave_g)
        push!(times_radon, time)
        
        # Save intermediate results after each iteration (in case of interruption)
        _dir = joinpath("output_$(SESSION)", "radon", "mcmc")
        isdir(_dir) || mkpath(_dir) # Ensure path is present
        save(joinpath(_dir, "times_naive-lgo.jld"), "data", times_radon)
    end
end

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter C:\Users\o6m1g\.julia\packages\ProgressMeter\kniaW\src\ProgressMeter.jl:607
Sampling 100%|███████████████████████████████| Time: 0:11:14
                                    iterations: 4000
                   ratio_divergent_transitions: 0.0
   ratio_divergent_transitions_during_adaption: 0.01
                                       n_steps: 15
                                     is_accept: true
                               acceptance_rate: 0.9614099185984277
                                   log_density: -959.3920399346343
                            hamiltonian_energy: 1053.451584473892
                      hamiltonian_energy_error: -0.11541114387546259

# LGO

In [11]:
chain = load(joinpath("output", "radon", "mcmc", "chain.jld"))["data"];
stats = CSV.read(joinpath("output", "radon", "mcmc", "stats_df.csv"), DataFrame);

## aSMC

In [12]:
function SMCS_Radon(chain::Vector{Vector{Float64}}, leave_g::Int64; k̂_threshold::Float64=0.7)
    
    # Unconstrain initial draws
    model   = Model(data, leave_g, 0) # draws are from this baseline
    draws_0 = chain .|> Turing.bijector(model) |> vecvec2mat # as matrix

    # Define problem dimensions
    R      = size(draws_0, 1)  # number of baseline MCMC draws
    D      = size(draws_0, 2)  # dimension of posterior
    L      = data.N_g[leave_g] # max number of intermediate distributions
    
    # Obtain unconstrained prior draw
    _names = sample(model, Prior(), 1).name_map.parameters .|> String # param names
    Θ_0    = NamedArray(
        draws_0,
        (1:R, _names),
        (:n, :d),
    )
    
    # Allocate storage
    particles = NamedArray(zeros(L+1, R, D), (0:L, 1:R, _names), (:l, :n, :d))
    weights   = NamedArray(zeros(L+1, R), (0:L, 1:R), (:l, :n))
    ϕ_history = NamedArray(zeros(L+1), 0:L, :l)
    mcmc_flag = NamedArray(zeros(L+1), 0:L, :l)
    k̂         = nothing
    
    # Log weights normalizer
    function _normalize(_log_w::Vector{Float64})::Vector{Float64}
        _w = exp.(_log_w .- maximum(_log_w))
        _w = _w / sum(_w)
        _w
    end
    
    # Effective sample size
    function _ess(_w::Vector{Float64})::Float64
        1 / sum(@. exp(2 * log(_w)))
    end
    
    # Log ratio at given proposal l-1 and target l for n-th particle
    function log_G(ℓπ_0, ℓπ_1, l::Int, n::Int)::Float64
        _particle = particles[:l => l-1, :n => n]
        log_γ_0 = LogDensityProblems.logdensity(ℓπ_0, _particle)
        log_γ_1 = LogDensityProblems.logdensity(ℓπ_1, _particle)
        log_γ_1 - log_γ_0
    end
    
    # MCMC kernel
    initial_ϵ, n_steps, mass = stats.step_size[end], min(15, stats.n_steps[end]), vec(var(chain .|> vec |> vecvec2mat; dims=1))
    function _move(initial_θ::Vector{Float64},
            n_samples::Int, n_adapts::Int,
            ℓπ::LogDensityFunction,
        )::Vector{Float64}
        
        metric = DiagEuclideanMetric(mass)
        hamiltonian = Hamiltonian(metric, ℓπ)
        integrator = Leapfrog(initial_ϵ)
        adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.8, integrator))
        kernel = HMCKernel(Trajectory{EndPointTS}(integrator, FixedNSteps(n_steps)))
        
        rng = Random.TaskLocalRNG()
        samples, stats = sample(rng, hamiltonian, kernel, initial_θ, n_samples, adaptor, n_adapts;
            verbose=false, progress=false)
        
        samples[end]
    end
    
    # Set initial values, starting index
    particles[:l => 0] = Θ_0
    weights[:l => 0]   = repeat([1/R], R)
    ϕ_history[:l => 0] = 0.
    
    time = @elapsed for l in 1:L
        # Inherit case-deleted model from previous iteration
        model_0 = Model(data, leave_g, ϕ_history[:l => l-1])
        ℓπ_0 = LogDensityFunction(model_0)
        Turing.DynamicPPL.link!!(ℓπ_0.varinfo, model_0)

        # Define (until ↦ ESS) map
        function _ϕ2reff(ϕ::Union{Float64, Int64})::Float64
            @assert 0 ≤ ϕ ≤ 1
            model_1 = Model(data, leave_g, ϕ)
            ℓπ_1 = LogDensityFunction(model_1)
            Turing.DynamicPPL.link!!(ℓπ_1.varinfo, model_1)
            
            _log_weights = [log_G(ℓπ_0, ℓπ_1, l, n) for n in 1:R]
            _log_weights = _log_weights .- maximum(_log_weights)
            
            _ESS = _log_weights |> _normalize |> _ess
            _ESS - R/2
        end
        
        # Initialize next distribution parameter
        ϕ_1 = nothing
        
        # Case 1: ESS is above threshold
        if _ϕ2reff(1) > 0
            # Define next as final distribution
            ϕ_1 = 1
            
            # Compute log weights
            model_1 = Model(data, leave_g, ϕ_1)
            ℓπ_1 = LogDensityFunction(model_1)
            Turing.DynamicPPL.link!!(ℓπ_1.varinfo, model_1)
            _log_weights = [log_G(ℓπ_0, ℓπ_1, l, n) for n in 1:R]
            _log_weights = _log_weights .- maximum(_log_weights)

            # Generalized Pareto
            _psis = psis(_log_weights; warn=false)
            
            # Case 1-1: PSIS
            if _psis.pareto_shape < k̂_threshold
                mcmc_flag[:l => l] = false; @info "ϕ=$(ϕ_1), PSIS"
                particles[:l => l] = particles[:l => l-1]
                weights[:l => l] = _psis.log_weights |> _normalize
                k̂ = _psis.pareto_shape 
                ϕ_history[:l => l] = ϕ_1
            
            # Case 1-2: MCMC kernel
            else
                mcmc_flag[:l => l] = true; @info "ϕ=$(ϕ_1), MCMC"
                
                A_0 = wsample(1:R, _log_weights |> _normalize, R)
                _begin = particles[:l => l-1, :n => A_0]
                _end = particles[:l => l, :n => 1:R] |> similar
                #@showprogress for n in 1:R
                p = ProgressMeter.Progress(R); @Threads.threads for n in 1:R
                    _end[n,:] = _move(_begin[n,:], 1, 0, ℓπ_1)
                    ProgressMeter.next!(p)
                end
                particles[Name(l), :, :,] = _end
                weights[:l => l] = repeat([1/R], R)
                ϕ_history[:l => l] = ϕ_1
            end
            
            L = l; break # Break loop

        # Case 2: ESS is below threshold
        else
            mcmc_flag[:l => l] = true
            
            # Find next distribution
            ϕ_1 = find_zero(_ϕ2reff, (ϕ_history[:l => l-1] + 1e-4, 1),
                xtol=0.1, maxiters=10, verbose=true)
            @info "Targeting ϕ=$(ϕ_1)"
            
            # Compute log weights
            model_1 = Model(data, leave_g, ϕ_1)
            ℓπ_1 = LogDensityFunction(model_1)
            Turing.DynamicPPL.link!!(ℓπ_1.varinfo, model_1)
            _log_weights = [log_G(ℓπ_0, ℓπ_1, l, n) for n in 1:R]
            _log_weights = _log_weights .- maximum(_log_weights)

            # MCMC kernel
            A_0 = wsample(1:R, _log_weights |> _normalize, R)
            _begin = particles[:l => l-1, :n => A_0]
            _end = particles[:l => l, :n => 1:R] |> similar
            #@showprogress for n in 1:R
            p = ProgressMeter.Progress(R); @Threads.threads for n in 1:R
                _end[n,:] = _move(_begin[n,:], 3, 0, ℓπ_1)
                ProgressMeter.next!(p)
            end
            particles[Name(l), :, :,] = _end
            weights[:l => l] = repeat([1/R], R)
            ϕ_history[:l => l] = ϕ_1
        end
    end

    particles = particles[Name(L),:,:]
    weights   = weights[Name(L), :]
    mcmc_flag = mcmc_flag[Name(L)]
    ϕ_history = ϕ_history[Name.(0:L)]

    (;
        R,         # Number of particles
        D,         # Dimensionality of parameter space
        L,         # Number of intermediate distributions
        particles, # NamedArray of particles
        weights,   # Associated particle weights
        ϕ_history, # History of ϕ (= 1 - ρ | ρ: power coef) values
        k̂,         # Pareto shape diagnostic estimate, if any
        mcmc_flag, # true if invariant kernel applied
        time,      # SMC runtime
    )
end

SMCS_Radon (generic function with 1 method)

## PSIS

In [13]:
function PSIS_Radon(chain::Vector{Vector{Float64}}, leave_g::Int64; k̂_threshold::Float64=0.7)

    # Unconstrain initial draws
    model   = Model(data, leave_g, 0) # draws are from this baseline
    draws_0 = chain .|> Turing.bijector(model) |> vecvec2mat # as matrix

    # Define problem dimensions
    R      = size(draws_0, 1)  # number of baseline MCMC draws
    D      = size(draws_0, 2)  # dimension of posterior
    L      = 1                 # max number of intermediate distributions
    
    # Obtain unconstrained prior draw
    _names = sample(model, Prior(), 1).name_map.parameters .|> String # param names
    Θ_0    = NamedArray(
        draws_0,
        (1:R, _names),
        (:n, :d),
    )
    
    # Allocate storage
    particles = NamedArray(zeros(L+1, R, D), (0:L, 1:R, _names), (:l, :n, :d))
    weights   = NamedArray(zeros(L+1, R), (0:L, 1:R), (:l, :n))
    k̂         = nothing

    # Log weights normalizer
    function _normalize(_log_w::Vector{Float64})::Vector{Float64}
        _w = exp.(_log_w .- maximum(_log_w))
        _w = _w / sum(_w)
        _w
    end
    
    # Log ratio at given proposal l-1 and target l for n-th particle
    function log_G(ℓπ_0, ℓπ_1, l::Int, n::Int)::Float64
        _particle = particles[:l => l-1, :n => n]
        log_γ_0 = LogDensityProblems.logdensity(ℓπ_0, _particle)
        log_γ_1 = LogDensityProblems.logdensity(ℓπ_1, _particle)
        log_γ_1 - log_γ_0
    end

    # Set initial values, starting index
    particles[:l => 0] = Θ_0
    weights[:l => 0]   = repeat([1/R], R)
    
    time = @elapsed (l = 1;
        ϕ_0 = 0;
        model_0 = Model(data, leave_g, ϕ_0);
        ℓπ_0 = LogDensityFunction(model_0);
        Turing.DynamicPPL.link!!(ℓπ_0.varinfo, model_0);
        
        # Compute log weights
        ϕ_1 = 1;
        model_1 = Model(data, leave_g, ϕ_1);
        ℓπ_1 = LogDensityFunction(model_1);
        Turing.DynamicPPL.link!!(ℓπ_1.varinfo, model_1);
        
        _log_weights = [log_G(ℓπ_0, ℓπ_1, l, n) for n in 1:R];
        _log_weights = _log_weights .- maximum(_log_weights);

        # PSIS
        _psis = psis(_log_weights; warn=false);
        particles[:l => l] = particles[:l => l-1];
        weights[:l => l] = _psis.log_weights |> _normalize;
    )
    
    k̂ = _psis.pareto_shape
    particles = particles[Name(L),:,:]
    weights   = weights[Name(L), :]

    (;
        R,         # Number of particles
        D,         # Dimensionality of parameter space
        L,         # = 1 distribution
        particles, # NamedArray of particles
        weights,   # Associated particle weights
        k̂,         # Pareto shape diagnostic estimate
        time,      # Runtime
    )
end

PSIS_Radon (generic function with 1 method)

# Run

In [56]:
# Function to run aSMC-LGO-CV or PSIS-LGO-CV across all counties
function run(
        use_chain::Vector{Vector{Float64}}, # Input baseline MCMC chain
        Sampler_Radon::Union{typeof(SMCS_Radon), typeof(PSIS_Radon)};
        save_bool::Bool=false,              # Saving output
        k̂_threshold::Float64=0.7,           # Optional threshold (unused for main analysis)
)
    
    R = length(use_chain) # Number of posterior particles

    # Init per-group storage
    particles    = []
    weights      = []
    ϕ_histories  = []
    k̂s           = []
    times        = Float64[]

    @info "First pilot run"
    Sampler_Radon(use_chain, 1; k̂_threshold=k̂_threshold)
    
    for leave_g in 1:data.G # Leave-out one group at a time
        @info "Leaving g=$(leave_g) out"
        
        # Run aSMC or PSIS
        results = Sampler_Radon(use_chain, leave_g; k̂_threshold=k̂_threshold)
        
        # Record output
        push!(particles,   results[:particles])
        push!(weights,     results[:weights])
        push!(k̂s,          results[:k̂])
        push!(times,       results[:time])
        Sampler_Radon == SMCS_Radon && push!(ϕ_histories, results[:ϕ_history])
        #break
    end

    if save_bool
        # Define folder to save output
        _dir = joinpath(
            "output_$(SESSION)",
            "radon",
            Sampler_Radon == SMCS_Radon ? "smc" : "psis"
        )
        isdir(_dir) || mkpath(_dir) # Ensure path is present

        # Save (after un-naming)
        # Sizes are: G = groups, R = particles, D = parameter dim
        save(joinpath(_dir, "particles_threshold-$(k̂_threshold).jld"),
            "data", particles .|> Matrix) # [G][R,D]
        save(joinpath(_dir, "weights_threshold-$(k̂_threshold).jld"),
            "data", weights .|> Vector) # [G][R]
        save(joinpath(_dir, "khats_threshold-$(k̂_threshold).jld"),
            "data", k̂s) # G
        save(joinpath(_dir, "times_threshold-$(k̂_threshold).jld"),
            "data", times) # G
        Sampler_Radon == SMCS_Radon && save(
            joinpath(_dir, "histories_threshold-$(k̂_threshold).jld"),
            "data", ϕ_histories .|> Vector) # [G][L_g]
    end
    
    return
end

run (generic function with 1 method)

In [57]:
Random.seed!(1)
RUN_aSMC && run(chain, SMCS_Radon; save_bool=true)

[ Info: Leaving g=1 out


Results of univariate zero finding:

* Converged to: 0.6313816406249998
* Algorithm: Bisection()
* iterations: 10
* function evaluations ≈ 13

Trace:
(a₀, b₀) = ( 0.010306249999999999, 1 )
(a₁, b₁) = ( 0.103725, 1 )
(a₂, b₂) = ( 0.33244999999999997, 1 )
(a₃, b₃) = ( 0.58244999999999991, 1 )
(a₄, b₄) = ( 0.58244999999999991, 0.79122499999999996 )
(a₅, b₅) = ( 0.58244999999999991, 0.68683749999999988 )
(a₆, b₆) = ( 0.58244999999999991, 0.63464374999999984 )
(a₇, b₇) = ( 0.60854687499999982, 0.63464374999999984 )
(a₈, b₈) = ( 0.62159531249999977, 0.63464374999999984 )
(a₉, b₉) = ( 0.62811953124999975, 0.63464374999999984 )
(a₁₀, b₁₀) = ( 0.62811953124999975, 0.6313816406249998 )



[ Info: Targeting ϕ=0.6313816406249998
Progress: 100%|█████████████████████████████████████████| Time: 0:03:19
[ Info: ϕ=1, PSIS
[ Info: Leaving g=2 out


Results of univariate zero finding:

* Converged to: 0.7325070312499998
* Algorithm: Bisection()
* iterations: 10
* function evaluations ≈ 13

Trace:
(a₀, b₀) = ( 0.010306249999999999, 1 )
(a₁, b₁) = ( 0.103725, 1 )
(a₂, b₂) = ( 0.33244999999999997, 1 )
(a₃, b₃) = ( 0.58244999999999991, 1 )
(a₄, b₄) = ( 0.58244999999999991, 0.79122499999999996 )
(a₅, b₅) = ( 0.68683749999999988, 0.79122499999999996 )
(a₆, b₆) = ( 0.68683749999999988, 0.73903124999999992 )
(a₇, b₇) = ( 0.7129343749999999, 0.73903124999999992 )
(a₈, b₈) = ( 0.72598281249999985, 0.73903124999999992 )
(a₉, b₉) = ( 0.73250703124999983, 0.73903124999999992 )
(a₁₀, b₁₀) = ( 0.73250703124999983, 0.73576914062499987 )



[ Info: Targeting ϕ=0.7325070312499998
Progress: 100%|█████████████████████████████████████████| Time: 0:03:17
[ Info: ϕ=1, PSIS
[ Info: Leaving g=3 out


...

[ Info: Targeting ϕ=0.6803132812499998
Progress: 100%|█████████████████████████████████████████| Time: 0:03:09
[ Info: ϕ=1, PSIS
[ Info: Leaving g=77 out


Results of univariate zero finding:

* Converged to: 0.9249714843749999
* Algorithm: Bisection()
* iterations: 10
* function evaluations ≈ 13

Trace:
(a₀, b₀) = ( 0.010306249999999999, 1 )
(a₁, b₁) = ( 0.103725, 1 )
(a₂, b₂) = ( 0.33244999999999997, 1 )
(a₃, b₃) = ( 0.58244999999999991, 1 )
(a₄, b₄) = ( 0.79122499999999996, 1 )
(a₅, b₅) = ( 0.89561249999999992, 1 )
(a₆, b₆) = ( 0.89561249999999992, 0.94780624999999996 )
(a₇, b₇) = ( 0.92170937499999994, 0.94780624999999996 )
(a₈, b₈) = ( 0.92170937499999994, 0.9347578124999999 )
(a₉, b₉) = ( 0.92170937499999994, 0.92823359374999992 )
(a₁₀, b₁₀) = ( 0.92497148437499987, 0.92823359374999992 )



[ Info: Targeting ϕ=0.9249714843749999
Progress: 100%|█████████████████████████████████████████| Time: 0:03:18
[ Info: ϕ=1, PSIS
[ Info: Leaving g=78 out


Results of univariate zero finding:

* Converged to: 0.9184472656249999
* Algorithm: Bisection()
* iterations: 10
* function evaluations ≈ 13

Trace:
(a₀, b₀) = ( 0.010306249999999999, 1 )
(a₁, b₁) = ( 0.103725, 1 )
(a₂, b₂) = ( 0.33244999999999997, 1 )
(a₃, b₃) = ( 0.58244999999999991, 1 )
(a₄, b₄) = ( 0.79122499999999996, 1 )
(a₅, b₅) = ( 0.89561249999999992, 1 )
(a₆, b₆) = ( 0.89561249999999992, 0.94780624999999996 )
(a₇, b₇) = ( 0.89561249999999992, 0.92170937499999994 )
(a₈, b₈) = ( 0.90866093749999988, 0.92170937499999994 )
(a₉, b₉) = ( 0.91518515624999985, 0.92170937499999994 )
(a₁₀, b₁₀) = ( 0.9184472656249999, 0.92170937499999994 )



[ Info: Targeting ϕ=0.9184472656249999
Progress: 100%|█████████████████████████████████████████| Time: 0:03:11
[ Info: ϕ=1, PSIS
[ Info: Leaving g=79 out
[ Info: ϕ=1, PSIS
[ Info: Leaving g=80 out
[ Info: ϕ=1, PSIS
[ Info: Leaving g=81 out
[ Info: ϕ=1, PSIS
[ Info: Leaving g=82 out
[ Info: ϕ=1, PSIS
[ Info: Leaving g=83 out
[ Info: ϕ=1, PSIS
[ Info: Leaving g=84 out
[ Info: ϕ=1, PSIS
[ Info: Leaving g=85 out
[ Info: ϕ=1, PSIS


In [58]:
Random.seed!(1)
RUN_aSMC && run(chain, PSIS_Radon; save_bool=true)

[ Info: Leaving g=1 out
[ Info: Leaving g=2 out
[ Info: Leaving g=3 out
[ Info: Leaving g=4 out
[ Info: Leaving g=5 out
[ Info: Leaving g=6 out
[ Info: Leaving g=7 out
[ Info: Leaving g=8 out
[ Info: Leaving g=9 out
[ Info: Leaving g=10 out
[ Info: Leaving g=11 out
[ Info: Leaving g=12 out
[ Info: Leaving g=13 out
[ Info: Leaving g=14 out
[ Info: Leaving g=15 out
[ Info: Leaving g=16 out
[ Info: Leaving g=17 out
[ Info: Leaving g=18 out
[ Info: Leaving g=19 out
[ Info: Leaving g=20 out
[ Info: Leaving g=21 out
[ Info: Leaving g=22 out
[ Info: Leaving g=23 out
[ Info: Leaving g=24 out
[ Info: Leaving g=25 out
[ Info: Leaving g=26 out
[ Info: Leaving g=27 out
[ Info: Leaving g=28 out
[ Info: Leaving g=29 out
[ Info: Leaving g=30 out
[ Info: Leaving g=31 out
[ Info: Leaving g=32 out
[ Info: Leaving g=33 out
[ Info: Leaving g=34 out
[ Info: Leaving g=35 out
[ Info: Leaving g=36 out
[ Info: Leaving g=37 out
[ Info: Leaving g=38 out
[ Info: Leaving g=39 out
[ Info: Leaving g=40 out
[ Info: L

## Sensitivity analyses

In [65]:
Random.seed!(1)
RUN_aSMC && run(chain, SMCS_Radon; k̂_threshold=0.9, save_bool=true)

[ Info: Leaving g=1 out


Results of univariate zero finding:

* Converged to: 0.6313816406249998
* Algorithm: Bisection()
* iterations: 10
* function evaluations ≈ 13

Trace:
(a₀, b₀) = ( 0.010306249999999999, 1 )
(a₁, b₁) = ( 0.103725, 1 )
(a₂, b₂) = ( 0.33244999999999997, 1 )
(a₃, b₃) = ( 0.58244999999999991, 1 )
(a₄, b₄) = ( 0.58244999999999991, 0.79122499999999996 )
(a₅, b₅) = ( 0.58244999999999991, 0.68683749999999988 )
(a₆, b₆) = ( 0.58244999999999991, 0.63464374999999984 )
(a₇, b₇) = ( 0.60854687499999982, 0.63464374999999984 )
(a₈, b₈) = ( 0.62159531249999977, 0.63464374999999984 )
(a₉, b₉) = ( 0.62811953124999975, 0.63464374999999984 )
(a₁₀, b₁₀) = ( 0.62811953124999975, 0.6313816406249998 )



[ Info: Targeting ϕ=0.6313816406249998
Progress: 100%|█████████████████████████████████████████| Time: 0:03:15
[ Info: ϕ=1, PSIS
[ Info: Leaving g=2 out


Results of univariate zero finding:

* Converged to: 0.7325070312499998
* Algorithm: Bisection()
* iterations: 10
* function evaluations ≈ 13

Trace:
(a₀, b₀) = ( 0.010306249999999999, 1 )
(a₁, b₁) = ( 0.103725, 1 )
(a₂, b₂) = ( 0.33244999999999997, 1 )
(a₃, b₃) = ( 0.58244999999999991, 1 )
(a₄, b₄) = ( 0.58244999999999991, 0.79122499999999996 )
(a₅, b₅) = ( 0.68683749999999988, 0.79122499999999996 )
(a₆, b₆) = ( 0.68683749999999988, 0.73903124999999992 )
(a₇, b₇) = ( 0.7129343749999999, 0.73903124999999992 )
(a₈, b₈) = ( 0.72598281249999985, 0.73903124999999992 )
(a₉, b₉) = ( 0.73250703124999983, 0.73903124999999992 )
(a₁₀, b₁₀) = ( 0.73250703124999983, 0.73576914062499987 )



[ Info: Targeting ϕ=0.7325070312499998
Progress: 100%|█████████████████████████████████████████| Time: 0:03:15
[ Info: ϕ=1, PSIS
[ Info: Leaving g=3 out


...

[ Info: Targeting ϕ=0.6803132812499998
Progress: 100%|█████████████████████████████████████████| Time: 0:03:11
[ Info: ϕ=1, PSIS
[ Info: Leaving g=77 out


Results of univariate zero finding:

* Converged to: 0.9249714843749999
* Algorithm: Bisection()
* iterations: 10
* function evaluations ≈ 13

Trace:
(a₀, b₀) = ( 0.010306249999999999, 1 )
(a₁, b₁) = ( 0.103725, 1 )
(a₂, b₂) = ( 0.33244999999999997, 1 )
(a₃, b₃) = ( 0.58244999999999991, 1 )
(a₄, b₄) = ( 0.79122499999999996, 1 )
(a₅, b₅) = ( 0.89561249999999992, 1 )
(a₆, b₆) = ( 0.89561249999999992, 0.94780624999999996 )
(a₇, b₇) = ( 0.92170937499999994, 0.94780624999999996 )
(a₈, b₈) = ( 0.92170937499999994, 0.9347578124999999 )
(a₉, b₉) = ( 0.92170937499999994, 0.92823359374999992 )
(a₁₀, b₁₀) = ( 0.92497148437499987, 0.92823359374999992 )



[ Info: Targeting ϕ=0.9249714843749999
Progress: 100%|█████████████████████████████████████████| Time: 0:03:05
[ Info: ϕ=1, PSIS
[ Info: Leaving g=78 out


Results of univariate zero finding:

* Converged to: 0.9184472656249999
* Algorithm: Bisection()
* iterations: 10
* function evaluations ≈ 13

Trace:
(a₀, b₀) = ( 0.010306249999999999, 1 )
(a₁, b₁) = ( 0.103725, 1 )
(a₂, b₂) = ( 0.33244999999999997, 1 )
(a₃, b₃) = ( 0.58244999999999991, 1 )
(a₄, b₄) = ( 0.79122499999999996, 1 )
(a₅, b₅) = ( 0.89561249999999992, 1 )
(a₆, b₆) = ( 0.89561249999999992, 0.94780624999999996 )
(a₇, b₇) = ( 0.89561249999999992, 0.92170937499999994 )
(a₈, b₈) = ( 0.90866093749999988, 0.92170937499999994 )
(a₉, b₉) = ( 0.91518515624999985, 0.92170937499999994 )
(a₁₀, b₁₀) = ( 0.9184472656249999, 0.92170937499999994 )



[ Info: Targeting ϕ=0.9184472656249999
Progress: 100%|█████████████████████████████████████████| Time: 0:03:11
[ Info: ϕ=1, PSIS
[ Info: Leaving g=79 out
[ Info: ϕ=1, PSIS
[ Info: Leaving g=80 out
[ Info: ϕ=1, PSIS
[ Info: Leaving g=81 out
[ Info: ϕ=1, PSIS
[ Info: Leaving g=82 out
[ Info: ϕ=1, PSIS
[ Info: Leaving g=83 out
[ Info: ϕ=1, PSIS
[ Info: Leaving g=84 out
[ Info: ϕ=1, PSIS
[ Info: Leaving g=85 out
[ Info: ϕ=1, PSIS


In [76]:
Random.seed!(1)
RUN_aSMC && run(chain, SMCS_Radon; k̂_threshold=0.5, save_bool=true)

[ Info: Leaving g=1 out


Results of univariate zero finding:

* Converged to: 0.6313816406249998
* Algorithm: Bisection()
* iterations: 10
* function evaluations ≈ 13

Trace:
(a₀, b₀) = ( 0.010306249999999999, 1 )
(a₁, b₁) = ( 0.103725, 1 )
(a₂, b₂) = ( 0.33244999999999997, 1 )
(a₃, b₃) = ( 0.58244999999999991, 1 )
(a₄, b₄) = ( 0.58244999999999991, 0.79122499999999996 )
(a₅, b₅) = ( 0.58244999999999991, 0.68683749999999988 )
(a₆, b₆) = ( 0.58244999999999991, 0.63464374999999984 )
(a₇, b₇) = ( 0.60854687499999982, 0.63464374999999984 )
(a₈, b₈) = ( 0.62159531249999977, 0.63464374999999984 )
(a₉, b₉) = ( 0.62811953124999975, 0.63464374999999984 )
(a₁₀, b₁₀) = ( 0.62811953124999975, 0.6313816406249998 )



[ Info: Targeting ϕ=0.6313816406249998
Progress: 100%|█████████████████████████████████████████| Time: 0:03:10
[ Info: ϕ=1, PSIS
[ Info: Leaving g=2 out


Results of univariate zero finding:

* Converged to: 0.7325070312499998
* Algorithm: Bisection()
* iterations: 10
* function evaluations ≈ 13

Trace:
(a₀, b₀) = ( 0.010306249999999999, 1 )
(a₁, b₁) = ( 0.103725, 1 )
(a₂, b₂) = ( 0.33244999999999997, 1 )
(a₃, b₃) = ( 0.58244999999999991, 1 )
(a₄, b₄) = ( 0.58244999999999991, 0.79122499999999996 )
(a₅, b₅) = ( 0.68683749999999988, 0.79122499999999996 )
(a₆, b₆) = ( 0.68683749999999988, 0.73903124999999992 )
(a₇, b₇) = ( 0.7129343749999999, 0.73903124999999992 )
(a₈, b₈) = ( 0.72598281249999985, 0.73903124999999992 )
(a₉, b₉) = ( 0.73250703124999983, 0.73903124999999992 )
(a₁₀, b₁₀) = ( 0.73250703124999983, 0.73576914062499987 )



[ Info: Targeting ϕ=0.7325070312499998
Progress: 100%|█████████████████████████████████████████| Time: 0:03:14
[ Info: ϕ=1, PSIS
[ Info: Leaving g=3 out


...

[ Info: Targeting ϕ=0.6803132812499998
Progress: 100%|█████████████████████████████████████████| Time: 0:03:12
[ Info: ϕ=1, PSIS
[ Info: Leaving g=77 out


Results of univariate zero finding:

* Converged to: 0.9249714843749999
* Algorithm: Bisection()
* iterations: 10
* function evaluations ≈ 13

Trace:
(a₀, b₀) = ( 0.010306249999999999, 1 )
(a₁, b₁) = ( 0.103725, 1 )
(a₂, b₂) = ( 0.33244999999999997, 1 )
(a₃, b₃) = ( 0.58244999999999991, 1 )
(a₄, b₄) = ( 0.79122499999999996, 1 )
(a₅, b₅) = ( 0.89561249999999992, 1 )
(a₆, b₆) = ( 0.89561249999999992, 0.94780624999999996 )
(a₇, b₇) = ( 0.92170937499999994, 0.94780624999999996 )
(a₈, b₈) = ( 0.92170937499999994, 0.9347578124999999 )
(a₉, b₉) = ( 0.92170937499999994, 0.92823359374999992 )
(a₁₀, b₁₀) = ( 0.92497148437499987, 0.92823359374999992 )



[ Info: Targeting ϕ=0.9249714843749999
Progress: 100%|█████████████████████████████████████████| Time: 0:03:14
[ Info: ϕ=1, PSIS
[ Info: Leaving g=78 out


Results of univariate zero finding:

* Converged to: 0.9184472656249999
* Algorithm: Bisection()
* iterations: 10
* function evaluations ≈ 13

Trace:
(a₀, b₀) = ( 0.010306249999999999, 1 )
(a₁, b₁) = ( 0.103725, 1 )
(a₂, b₂) = ( 0.33244999999999997, 1 )
(a₃, b₃) = ( 0.58244999999999991, 1 )
(a₄, b₄) = ( 0.79122499999999996, 1 )
(a₅, b₅) = ( 0.89561249999999992, 1 )
(a₆, b₆) = ( 0.89561249999999992, 0.94780624999999996 )
(a₇, b₇) = ( 0.89561249999999992, 0.92170937499999994 )
(a₈, b₈) = ( 0.90866093749999988, 0.92170937499999994 )
(a₉, b₉) = ( 0.91518515624999985, 0.92170937499999994 )
(a₁₀, b₁₀) = ( 0.9184472656249999, 0.92170937499999994 )



[ Info: Targeting ϕ=0.9184472656249999
Progress: 100%|█████████████████████████████████████████| Time: 0:03:14
[ Info: ϕ=1, PSIS
[ Info: Leaving g=79 out
[ Info: ϕ=1, PSIS
[ Info: Leaving g=80 out
[ Info: ϕ=1, PSIS
[ Info: Leaving g=81 out
[ Info: ϕ=1, PSIS
[ Info: Leaving g=82 out
[ Info: ϕ=1, PSIS
[ Info: Leaving g=83 out
[ Info: ϕ=1, PSIS
[ Info: Leaving g=84 out
[ Info: ϕ=1, PSIS
[ Info: Leaving g=85 out
[ Info: ϕ=1, MCMC
Progress: 100%|█████████████████████████████████████████| Time: 0:01:07


In [82]:
Random.seed!(1)
RUN_aSMC && run(chain, SMCS_Radon; k̂_threshold=-Inf, save_bool=true)

[ Info: Leaving g=1 out


Results of univariate zero finding:

* Converged to: 0.6313816406249998
* Algorithm: Bisection()
* iterations: 10
* function evaluations ≈ 13

Trace:
(a₀, b₀) = ( 0.010306249999999999, 1 )
(a₁, b₁) = ( 0.103725, 1 )
(a₂, b₂) = ( 0.33244999999999997, 1 )
(a₃, b₃) = ( 0.58244999999999991, 1 )
(a₄, b₄) = ( 0.58244999999999991, 0.79122499999999996 )
(a₅, b₅) = ( 0.58244999999999991, 0.68683749999999988 )
(a₆, b₆) = ( 0.58244999999999991, 0.63464374999999984 )
(a₇, b₇) = ( 0.60854687499999982, 0.63464374999999984 )
(a₈, b₈) = ( 0.62159531249999977, 0.63464374999999984 )
(a₉, b₉) = ( 0.62811953124999975, 0.63464374999999984 )
(a₁₀, b₁₀) = ( 0.62811953124999975, 0.6313816406249998 )



[ Info: Targeting ϕ=0.6313816406249998
Progress: 100%|█████████████████████████████████████████| Time: 0:03:14
[ Info: ϕ=1, MCMC
Progress: 100%|█████████████████████████████████████████| Time: 0:01:07
[ Info: Leaving g=2 out


Results of univariate zero finding:

* Converged to: 0.7325070312499998
* Algorithm: Bisection()
* iterations: 10
* function evaluations ≈ 13

Trace:
(a₀, b₀) = ( 0.010306249999999999, 1 )
(a₁, b₁) = ( 0.103725, 1 )
(a₂, b₂) = ( 0.33244999999999997, 1 )
(a₃, b₃) = ( 0.58244999999999991, 1 )
(a₄, b₄) = ( 0.58244999999999991, 0.79122499999999996 )
(a₅, b₅) = ( 0.68683749999999988, 0.79122499999999996 )
(a₆, b₆) = ( 0.68683749999999988, 0.73903124999999992 )
(a₇, b₇) = ( 0.7129343749999999, 0.73903124999999992 )
(a₈, b₈) = ( 0.72598281249999985, 0.73903124999999992 )
(a₉, b₉) = ( 0.73250703124999983, 0.73903124999999992 )
(a₁₀, b₁₀) = ( 0.73250703124999983, 0.73576914062499987 )



[ Info: Targeting ϕ=0.7325070312499998
Progress: 100%|█████████████████████████████████████████| Time: 0:03:13
[ Info: ϕ=1, MCMC
Progress: 100%|█████████████████████████████████████████| Time: 0:01:08
[ Info: Leaving g=3 out


...

[ Info: Targeting ϕ=0.6803132812499998
Progress: 100%|█████████████████████████████████████████| Time: 0:03:17
[ Info: ϕ=1, MCMC
Progress: 100%|█████████████████████████████████████████| Time: 0:01:06
[ Info: Leaving g=77 out


Results of univariate zero finding:

* Converged to: 0.9249714843749999
* Algorithm: Bisection()
* iterations: 10
* function evaluations ≈ 13

Trace:
(a₀, b₀) = ( 0.010306249999999999, 1 )
(a₁, b₁) = ( 0.103725, 1 )
(a₂, b₂) = ( 0.33244999999999997, 1 )
(a₃, b₃) = ( 0.58244999999999991, 1 )
(a₄, b₄) = ( 0.79122499999999996, 1 )
(a₅, b₅) = ( 0.89561249999999992, 1 )
(a₆, b₆) = ( 0.89561249999999992, 0.94780624999999996 )
(a₇, b₇) = ( 0.92170937499999994, 0.94780624999999996 )
(a₈, b₈) = ( 0.92170937499999994, 0.9347578124999999 )
(a₉, b₉) = ( 0.92170937499999994, 0.92823359374999992 )
(a₁₀, b₁₀) = ( 0.92497148437499987, 0.92823359374999992 )



[ Info: Targeting ϕ=0.9249714843749999
Progress: 100%|█████████████████████████████████████████| Time: 0:03:13
[ Info: ϕ=1, MCMC
Progress: 100%|█████████████████████████████████████████| Time: 0:01:05
[ Info: Leaving g=78 out


Results of univariate zero finding:

* Converged to: 0.9184472656249999
* Algorithm: Bisection()
* iterations: 10
* function evaluations ≈ 13

Trace:
(a₀, b₀) = ( 0.010306249999999999, 1 )
(a₁, b₁) = ( 0.103725, 1 )
(a₂, b₂) = ( 0.33244999999999997, 1 )
(a₃, b₃) = ( 0.58244999999999991, 1 )
(a₄, b₄) = ( 0.79122499999999996, 1 )
(a₅, b₅) = ( 0.89561249999999992, 1 )
(a₆, b₆) = ( 0.89561249999999992, 0.94780624999999996 )
(a₇, b₇) = ( 0.89561249999999992, 0.92170937499999994 )
(a₈, b₈) = ( 0.90866093749999988, 0.92170937499999994 )
(a₉, b₉) = ( 0.91518515624999985, 0.92170937499999994 )
(a₁₀, b₁₀) = ( 0.9184472656249999, 0.92170937499999994 )



[ Info: Targeting ϕ=0.9184472656249999
Progress: 100%|█████████████████████████████████████████| Time: 0:03:12
[ Info: ϕ=1, MCMC
Progress: 100%|█████████████████████████████████████████| Time: 0:01:08
[ Info: Leaving g=79 out
[ Info: ϕ=1, MCMC
Progress: 100%|█████████████████████████████████████████| Time: 0:01:10
[ Info: Leaving g=80 out
[ Info: ϕ=1, MCMC
Progress: 100%|█████████████████████████████████████████| Time: 0:01:07
[ Info: Leaving g=81 out
[ Info: ϕ=1, MCMC
Progress: 100%|█████████████████████████████████████████| Time: 0:01:08
[ Info: Leaving g=82 out
[ Info: ϕ=1, MCMC
Progress: 100%|█████████████████████████████████████████| Time: 0:01:05
[ Info: Leaving g=83 out
[ Info: ϕ=1, MCMC
Progress: 100%|█████████████████████████████████████████| Time: 0:01:10
[ Info: Leaving g=84 out
[ Info: ϕ=1, MCMC
Progress: 100%|█████████████████████████████████████████| Time: 0:01:10
[ Info: Leaving g=85 out
[ Info: ϕ=1, MCMC
Progress: 100%|█████████████████████████████████████████| Time: 0:

In [88]:
Random.seed!(1)
RUN_aSMC && run(chain, SMCS_Radon; k̂_threshold=+Inf, save_bool=true)

[ Info: Leaving g=1 out


Results of univariate zero finding:

* Converged to: 0.6313816406249998
* Algorithm: Bisection()
* iterations: 10
* function evaluations ≈ 13

Trace:
(a₀, b₀) = ( 0.010306249999999999, 1 )
(a₁, b₁) = ( 0.103725, 1 )
(a₂, b₂) = ( 0.33244999999999997, 1 )
(a₃, b₃) = ( 0.58244999999999991, 1 )
(a₄, b₄) = ( 0.58244999999999991, 0.79122499999999996 )
(a₅, b₅) = ( 0.58244999999999991, 0.68683749999999988 )
(a₆, b₆) = ( 0.58244999999999991, 0.63464374999999984 )
(a₇, b₇) = ( 0.60854687499999982, 0.63464374999999984 )
(a₈, b₈) = ( 0.62159531249999977, 0.63464374999999984 )
(a₉, b₉) = ( 0.62811953124999975, 0.63464374999999984 )
(a₁₀, b₁₀) = ( 0.62811953124999975, 0.6313816406249998 )



[ Info: Targeting ϕ=0.6313816406249998
Progress: 100%|█████████████████████████████████████████| Time: 0:03:18
[ Info: ϕ=1, PSIS
[ Info: Leaving g=2 out


Results of univariate zero finding:

* Converged to: 0.7325070312499998
* Algorithm: Bisection()
* iterations: 10
* function evaluations ≈ 13

Trace:
(a₀, b₀) = ( 0.010306249999999999, 1 )
(a₁, b₁) = ( 0.103725, 1 )
(a₂, b₂) = ( 0.33244999999999997, 1 )
(a₃, b₃) = ( 0.58244999999999991, 1 )
(a₄, b₄) = ( 0.58244999999999991, 0.79122499999999996 )
(a₅, b₅) = ( 0.68683749999999988, 0.79122499999999996 )
(a₆, b₆) = ( 0.68683749999999988, 0.73903124999999992 )
(a₇, b₇) = ( 0.7129343749999999, 0.73903124999999992 )
(a₈, b₈) = ( 0.72598281249999985, 0.73903124999999992 )
(a₉, b₉) = ( 0.73250703124999983, 0.73903124999999992 )
(a₁₀, b₁₀) = ( 0.73250703124999983, 0.73576914062499987 )



[ Info: Targeting ϕ=0.7325070312499998
Progress: 100%|█████████████████████████████████████████| Time: 0:03:14
[ Info: ϕ=1, PSIS
[ Info: Leaving g=3 out


...

[ Info: Targeting ϕ=0.6803132812499998
Progress: 100%|█████████████████████████████████████████| Time: 0:03:05
[ Info: ϕ=1, PSIS
[ Info: Leaving g=77 out


Results of univariate zero finding:

* Converged to: 0.9249714843749999
* Algorithm: Bisection()
* iterations: 10
* function evaluations ≈ 13

Trace:
(a₀, b₀) = ( 0.010306249999999999, 1 )
(a₁, b₁) = ( 0.103725, 1 )
(a₂, b₂) = ( 0.33244999999999997, 1 )
(a₃, b₃) = ( 0.58244999999999991, 1 )
(a₄, b₄) = ( 0.79122499999999996, 1 )
(a₅, b₅) = ( 0.89561249999999992, 1 )
(a₆, b₆) = ( 0.89561249999999992, 0.94780624999999996 )
(a₇, b₇) = ( 0.92170937499999994, 0.94780624999999996 )
(a₈, b₈) = ( 0.92170937499999994, 0.9347578124999999 )
(a₉, b₉) = ( 0.92170937499999994, 0.92823359374999992 )
(a₁₀, b₁₀) = ( 0.92497148437499987, 0.92823359374999992 )



[ Info: Targeting ϕ=0.9249714843749999
Progress: 100%|█████████████████████████████████████████| Time: 0:03:04
[ Info: ϕ=1, PSIS
[ Info: Leaving g=78 out


Results of univariate zero finding:

* Converged to: 0.9184472656249999
* Algorithm: Bisection()
* iterations: 10
* function evaluations ≈ 13

Trace:
(a₀, b₀) = ( 0.010306249999999999, 1 )
(a₁, b₁) = ( 0.103725, 1 )
(a₂, b₂) = ( 0.33244999999999997, 1 )
(a₃, b₃) = ( 0.58244999999999991, 1 )
(a₄, b₄) = ( 0.79122499999999996, 1 )
(a₅, b₅) = ( 0.89561249999999992, 1 )
(a₆, b₆) = ( 0.89561249999999992, 0.94780624999999996 )
(a₇, b₇) = ( 0.89561249999999992, 0.92170937499999994 )
(a₈, b₈) = ( 0.90866093749999988, 0.92170937499999994 )
(a₉, b₉) = ( 0.91518515624999985, 0.92170937499999994 )
(a₁₀, b₁₀) = ( 0.9184472656249999, 0.92170937499999994 )



[ Info: Targeting ϕ=0.9184472656249999
Progress: 100%|█████████████████████████████████████████| Time: 0:03:17
[ Info: ϕ=1, PSIS
[ Info: Leaving g=79 out
[ Info: ϕ=1, PSIS
[ Info: Leaving g=80 out
[ Info: ϕ=1, PSIS
[ Info: Leaving g=81 out
[ Info: ϕ=1, PSIS
[ Info: Leaving g=82 out
[ Info: ϕ=1, PSIS
[ Info: Leaving g=83 out
[ Info: ϕ=1, PSIS
[ Info: Leaving g=84 out
[ Info: ϕ=1, PSIS
[ Info: Leaving g=85 out
[ Info: ϕ=1, PSIS


# Figures

## LPD

In [94]:
function Compute_LPDs(results, leave_g::Int64, _model, _names; constrain=true)::Vector{Float64}

    # compute log predictive density (LPD) for a single particle
    function LPD(particle)::Float64
        # Transform particle to named space for convenience
        particle = NamedArray(
            constrain ? particle |> Turing.inverse(Turing.bijector(_model)) : particle,
            _names, # Label parameters
            :d      # Use :d as axis name
        )
        β = particle[:d => ["β[$(leave_g)][$(k)]" for k in 1:2]]
        v_y = particle[:d => "v_y"]
        logpdf(MvNormal(X_g * β, v_y * I), y_g)
    end

    # Subset y and X for observations in group leave_g
    leave_g_idx = data.county .== leave_g
    y_g, X_g = data.y[leave_g_idx], data.X[leave_g_idx,:]
    @assert data.N_g[leave_g] == sum(leave_g_idx) # Assert size matches known count

    # Compute LPDs for each posterior particle
    LPDs = Float64[]
    for n in 1:results.R
        push!(LPDs, results[:particles][:n => n] |> LPD)
    end
    
    LPDs
end

Compute_LPDs (generic function with 1 method)

In [95]:
k̂_threshold = .7

0.7

In [96]:
let
    # Prepare dimensions and containers
    R                   = length(load(joinpath("output", "radon", "mcmc", "chain.jld"))["data"])
    LPD_template        = NamedArray(zeros(R, data.G), (1:R, 1:data.G), (:n, :g))
    global LPD_mcmc     = similar(LPD_template)
    global LPD_smc      = similar(LPD_template)
    global LPD_psis     = similar(LPD_template)
    global weights_smc  = load(joinpath("output", "radon", "smc",
            "weights_threshold-$(k̂_threshold).jld"))["data"]
    global weights_psis = load(joinpath("output", "radon", "psis",
            "weights_threshold-$(0.7).jld"))["data"]
    
    # Helper to wrap chain into NamedArray for Compute_LPDs
    function _wrap_particles(_particles::Vector)
        R = length(_particles)
        results = (; particles=NamedArray(_particles, 1:R, :n), R=R)
    end

    # Pre-load/instantiate particles / baseline posterior
    chain_smc = load(joinpath("output", "radon", "smc",
            "particles_threshold-$(k̂_threshold).jld"))["data"]
    chain_psis = load(joinpath("output", "radon", "psis",
            "particles_threshold-$(0.7).jld"))["data"]
    _model = Model(data, -1, 0) # leave_g = -1 to disable leave-group-out logic
    _names = sample(_model, Prior(), 1).name_map.parameters .|> String # Parameter names
    
    # Loop over all groups
    @info "Computing LPDs."; sleep(0.01)
    @showprogress for leave_g in 1:data.G
        # @info "Leaving g=$(leave_g) out"
        
        # MCMC as ground truth
        chain = load(joinpath("output", "radon", "mcmc", "chain_lgo-$(leave_g).jld"))["data"]
        LPD_mcmc[:, leave_g] = Compute_LPDs(
            chain |> _wrap_particles,
            leave_g, _model, _names;
            constrain=false) # LGO draws are already constrained to original space
    
        # SMC estimate
        LPD_smc[:, leave_g] = Compute_LPDs(
            chain_smc[leave_g] |> mat2vecvec |> _wrap_particles,
            leave_g, _model, _names;
            constrain=true)
    
        # PSIS estimate
        LPD_psis[:, leave_g] = Compute_LPDs(
            chain_psis[leave_g] |> mat2vecvec |> _wrap_particles,
            leave_g, _model, _names;
            constrain=true)
    end
end

[ Info: Computing LPDs.
Progress: 100%|█████████████████████████████████████████| Time: 0:08:28


In [ ]:
let
    R, G = size(LPD_mcmc)
    equal_weights_cumsum = cumsum(fill(1/R, R))
    COUNTIES = unique(df.county)
    COUNTMAP = countmap(df.county)
    re_idx = sortperm([sum(df.county .== g) for g in COUNTIES], rev=true)
    
    # Plot
    n_rows = 2
    plts = []; for (g_i, g) in enumerate([1,2,3, data.G-2, data.G-1, data.G]) # 3 highest-/lowest-N groups 
        
        # Plot where PSIS distribution approximation "ends"
        vline([minimum(LPD_psis[:, re_idx[g]])], label="", ylim=(0,1),
            color=:green, alpha=0.75, linewidth=1, linestyle=:dash)

        # MCMC empirical cdf
        plot!(sort(LPD_mcmc[:, re_idx[g]]),
            equal_weights_cumsum,
            color=:blue, label="", linewidth=4, alpha=0.8)

        # SMC empirical cdf
        cdf_sort_idx = LPD_smc[:, re_idx[g]] |> sortperm
        plot!(LPD_smc[cdf_sort_idx, re_idx[g]],
            weights_smc[re_idx[g]][cdf_sort_idx] |> cumsum,
            color=:red, label="", linewidth=3, linestyle=:dot, alpha=.75)

        # PSIS empirical cdf
        cdf_sort_idx = LPD_psis[:, re_idx[g]] |> sortperm
        plot!(LPD_psis[cdf_sort_idx, re_idx[g]],
            weights_psis[re_idx[g]][cdf_sort_idx] |> cumsum,
            color=:green, label="", linewidth=2, linestyle=:dash, alpha=1)
        
        if g_i == 1
            annotate!(-157, 0.35, text("MCMC", :blue, 7))
            annotate!(-166, 0.20, text("SMC", :red, 7))
            annotate!(-144, 0.10, text("PSIS", :green, 7))
        end
        
        push!(plts, plot!(
                title="$(COUNTIES[re_idx[g]]) (" * L"N_g" * "=$(COUNTMAP[COUNTIES[re_idx[g]]]))",
                legend=g == 1 ? true : false,
                xlim=(LPD_smc[:, re_idx[g]] |> minimum, LPD_mcmc[:, re_idx[g]] |> maximum)))
    end
    
    plot(plts..., layout=(n_rows, 3), size=(600,150n_rows),
        tick_direction=:out,
        titlefontsize=11,
        ytickfontsize=7, yticks=0:0.25:1,
        xtickfontsize=6, xrot=30,
        topmargin=2Plots.mm, bottommargin=3Plots.mm,
        xlabel=["" "" "" "" "Log predictive density" ""],
    )# |> display

    _dir = "img_$(SESSION)" * (k̂_threshold == 0.7 ? "" : "_threshold-$(k̂_threshold)")
    isdir(_dir) || mkpath(_dir) # Ensure path is present
    savefig(joinpath(_dir, "radon-lpd-compare.pdf"))
end

## Distributions

In [ ]:
let
    R, G = size(LPD_mcmc)
    equal_weights_cumsum = cumsum(fill(1/R, R))
    COUNTIES = unique(df.county)
    COUNTMAP = countmap(df.county)
    re_idx = sortperm([sum(df.county .== g) for g in COUNTIES], rev=true)
    
    est_mcmc = _mean(LPD_mcmc; dims=1)[re_idx]
    est_smc  = _sum(vecvec2mat(weights_smc)' .* LPD_smc; dims=1)[re_idx]
    est_psis = _sum(vecvec2mat(weights_psis)' .* LPD_psis; dims=1)[re_idx]
    
    loss_smc = abs.(est_mcmc - est_smc)
    loss_psis = abs.(est_mcmc - est_psis)
    
    plot(data.N_g[re_idx], loss_smc, color=:red,
        ms=4, mswidth=0, label="", alpha=1, markershape=:o, linestyle=:solid)
    plot!(data.N_g[re_idx], loss_psis, color=:green,
        ms=4, mswidth=0, label="", alpha=0.5, markershape=:square, linestyle=:solid)

    for i in 1:6
        _x = data.N_g[re_idx[i]]
        annotate!(_x, loss_psis[i], text("$(COUNTIES[re_idx[i]])\n\n", 1 ≤ i ≤ 3 ? 7 : 6))

        if i == 1
            annotate!(_x + 3, loss_smc[i] + 0.4, text("SMC-LGO   ", 10, :right, :red))
            annotate!(_x + 3, loss_psis[i], text("\n\n\nPSIS-LGO   ", 10, :right, :green))
        end
    end
    plt_loss = plot!(xlim=(0, 1.1maximum(data.N_g)), ylim=(0, 1.1maximum([loss_psis; loss_smc])))

    # k̂_threshold = 0.7
    histories = load(joinpath("output", "radon", "smc", "histories_threshold-$(k̂_threshold).jld"))["data"]
    
    plot()
    select_idx = [1:6; -1; Int.(round.(LinRange(6, data.G, 4+1)[2:end]))]
    #for (g, (i, history)) in enumerate(zip(select_idx, histories[re_idx][select_idx]))
    for (i, g) in enumerate(select_idx)
        g == -1 && continue
        history = histories[re_idx[g]]
        for ℓ in history
            if 0 < ℓ < 1
                if g <= 2
                    annotate!(ℓ, i + 0.50, text("$(round(ℓ; digits=2))"[2:end], :right, :black, :bottom, 7, rotation=30))
                else
                    annotate!(ℓ, i + 0.5, text("$(round(ℓ; digits=2)) "[2:end], :right, :black, :center, 7))
                end
            end
        end
        scatter!(history, repeat([i], length(history)), ms=4, mswidth=0.1, color=:red, label="")
    end

    _offset = 0.000
    plot!(
        xlim=(0-_offset, 1+_offset),
        xticks=(
            [0, 0.5, 1],
            [L"\varphi = 0" * "\n(Baseline)", L"\rightarrow", L"\varphi = 1" * "\n(Leave-group-out)"]
        ),
        yticks=Tuple(eachcol(vcat([permutedims([i, prod([COUNTIES[re_idx[g]], " (", L"N_g", "=", "$(COUNTMAP[COUNTIES[re_idx[g]]]))"])]) for (i, g) in enumerate(select_idx) if g != -1]...))),
        ymirror=true, ytickfontsize=8,
        framestyle=:box,
        yflip=true
    )
    plt_dist = annotate!(1.3, 6+1, text(L"\vdots", 9))

    plot(plt_loss, plt_dist, layout=grid(1,2,widths=[0.6,0.4]), size=(650,250),
        title=["Absolute error" repeat(" ", 10) * "SMC-LGO adaptive paths"], #titlefontsize=14,
        xlabel=["Within-group observations " * L"N_g" ""], xlabelfontsize=10,
        rightmargin=[3Plots.mm 0Plots.mm], bottommargin=2Plots.mm,
        tick_direction=:out
    )# |> display

    _dir = "img_$(SESSION)" * (k̂_threshold == 0.7 ? "" : "_threshold-$(k̂_threshold)")
    isdir(_dir) || mkpath(_dir) # Ensure path is present
    savefig(joinpath(_dir, "radon-lpd-error.pdf"))
end

## Runtime

In [ ]:
let
    # k̂_threshold = 0.7
    time_mcmc = load(joinpath("output", "radon", "mcmc",
            "times_naive-lgo.jld"))["data"] / 60
    time_smc  = load(joinpath("output", "radon", "smc",
            "times_threshold-$(k̂_threshold).jld"))["data"] / 60
    time_psis = load(joinpath("output", "radon", "psis",
            "times_threshold-$(0.7).jld"))["data"] / 60
    
    re_idx = time_mcmc |> sortperm
    time_mcmc = time_mcmc[re_idx]
    time_smc  = time_smc[re_idx]
    time_psis = time_psis[re_idx]

    COUNTIES = unique(df.county)[re_idx]
    COUNTMAP = df.county |> countmap

    plot(time_mcmc, markershape=:o, ms=2, mswidth=0, color=:blue)
    plot!(time_psis, markershape=:o, ms=2, mswidth=0, color=:green)
    plot!(time_smc, markershape=:o, ms=2, mswidth=0, color=:red)
    
    annotate!(data.G + 1, time_mcmc[end], text("MCMC-LGO", :blue, :left, 9))
    annotate!(data.G + 1, time_smc[end] + 2, text("SMC-LGO", :red, :left, 9))
    annotate!(data.G + 1, time_psis[end] + 1.3, text("PSIS-LGO", :green, :left, 9))

    
    histories = load(joinpath("output", "radon", "smc",
            "histories_threshold-$(k̂_threshold).jld"))["data"]
    for (g, history) in enumerate(histories[re_idx])
        L_g = length(history) - 1
        if L_g == 1
            # annotate!(g, time_smc[g] - 0.1, text(L_g-1, :red, :top, 5))
        else
            annotate!(g, time_smc[g], text(L_g-1, :black, 7))
        end
    end
    plot!(tick_direction=:out,
        ylim=(0, maximum([time_mcmc; time_smc; time_psis])),
        xticks=sort(unique([0:5:data.G; 1; data.G][2:end])),
        xlabel="County (reordered)", title="Runtime [min.]",
        legend=false, size=(600, 250),
        rightmargin=15Plots.mm, bottommargin=3Plots.mm
    )
    
    _dir = "img_$(SESSION)" * (k̂_threshold == 0.7 ? "" : "_threshold-$(k̂_threshold)")
    isdir(_dir) || mkpath(_dir) # Ensure path is present
    savefig(joinpath(_dir, "radon-runtime.pdf"))
end